# K-mer based TF binding prediction using sklearn

This tutorial illustrates how sklearn can be used with Janggu datasets.

In particular, we shall use the same toy sequences as in the other tutorial. But this time we fit a few sklearn models to descriminate transcription factor binding events based on k-mer frequencies.

In [1]:
import os

import numpy as np


from pkg_resources import resource_filename

from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.data import SqueezeDim

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from IPython.display import Image

np.random.seed(1234)


/home/wkopp/miniconda3/envs/jdev/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.environ['JANGGU_OUTPUT'] = '/home/wkopp/janggu_examples'

You might want to play around with the orders, which corresponds to the k-mer length. For small choices e.g. k=1 or 2
the performance should end up to be lower than for longer kmers e.g. k=5.
However, be aware that too large numbers of k will amount to large memory usage. 

In [3]:
order = 5

In [4]:
# load the dataset
# The pseudo genome represents just a concatenation of all sequences
# in sample.fa and sample2.fa. Therefore, the results should be almost
# identically to the models obtained from classify_fasta.py.
REFGENOME = resource_filename('janggu', 'resources/pseudo_genome.fa')
# ROI contains regions spanning positive and negative examples
ROI_TRAIN_FILE = resource_filename('janggu', 'resources/roi_train.bed')
ROI_TEST_FILE = resource_filename('janggu', 'resources/roi_test.bed')
# PEAK_FILE only contains positive examples
PEAK_FILE = resource_filename('janggu', 'resources/scores.bed')

Now we obtain the k-mer representation for each sequences as well as the labels.

Note: the way sklearn utilizes the data requires to wrap the datasets via SqueezeDim, which is not necessary with keras.

In [5]:
# Training input and labels are purely defined genomic coordinates
DNA = SqueezeDim(ReduceDim(Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                   roi=ROI_TRAIN_FILE,
                                   binsize=200,
                                   order=order,
                                   cache=True)))

LABELS = SqueezeDim(ReduceDim(Cover.create_from_bed('peaks', roi=ROI_TRAIN_FILE,
                               bedfiles=PEAK_FILE,
                               binsize=200,
                               resolution=200,
                               cache=True,
                               storage='sparse')))


DNA_TEST = SqueezeDim(ReduceDim(Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                        roi=ROI_TEST_FILE,
                                        binsize=200,
                                        order=order)))

LABELS_TEST = SqueezeDim(ReduceDim(Cover.create_from_bed('peaks',
                                    bedfiles=PEAK_FILE,
                                    roi=ROI_TEST_FILE,
                                    binsize=200,
                                    resolution=200,
                                    storage='sparse')))


loading from lazy loader
reload /home/wkopp/janggu_examples/datasets/dna/aded3c22c15e2ececdbed2fdf4e19b145ab947eda546abd668b8ae2e76f787e5.npz
loading from bed lazy loader
reload /home/wkopp/janggu_examples/datasets/peaks/fd9826cf7fb9cc044a6c1354a14688c1be0f0bd9c593fdba2e9af3284a2be099.npz
loading from lazy loader
loading from bed lazy loader


k-mer count matrix used for training is shown below

In [6]:
DNA.shape

(7797, 1024)

we compare the performances for logistic regression, support vector machines and random forest.

In [7]:
logreg = LogisticRegression()
logreg.fit(DNA, LABELS)
logregpred = logreg.predict_proba(DNA_TEST)[:,1]

In [8]:
svc = SVC(probability=True)
svc.fit(DNA, LABELS)
svcpred = svc.predict_proba(DNA_TEST)[:,1]

In [9]:
rf = RandomForestClassifier()
rf.fit(DNA, LABELS)
rfpred = rf.predict_proba(DNA_TEST)[:,1]

Support vector machines outperforms logistic regression and random forest in this example.

In [10]:
print("AUC_LogReg: ", roc_auc_score(LABELS_TEST[:], logregpred))
print("AUC_SVM: ", roc_auc_score(LABELS_TEST[:], svcpred))
print("AUC_RF: ", roc_auc_score(LABELS_TEST[:], rfpred))

AUC_LogReg:  0.9055
AUC_SVM:  0.9357000000000001
AUC_RF:  0.8668
